In [1]:
!pip install schedule
!pip install pyti

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import time
import pytz
import shutil
import smtplib
import schedule
import numpy as np
import pandas as pd
import yfinance as yf
import pyti.bollinger_bands as bb
import pyti.money_flow_index as mfi
from email.mime.text import MIMEText
import pyti.hull_moving_average as hma
import pyti.simple_moving_average as sma
from IPython.display import clear_output
import pyti.relative_strength_index as rsi
import pyti.exponential_moving_average as ema
from email.mime.multipart import MIMEMultipart
from sklearn.neighbors import KNeighborsRegressor
from datetime import datetime, timedelta, timezone

In [3]:
# Constantes
window_sizes = {'upper_band': 20, 'middle_band': 20, 'lower_band': 20, 'ema_10': 10, 'ema_20': 20,'ema_30': 30,'ema_50': 50,'ema_100': 100, 'sma_10': 10,'sma_20': 20,'sma_30': 30,'sma_50': 50,'sma_100': 100, 'hma_9': 9}
weights = {'upper_band': 2,'middle_band': 2,'lower_band': 2, 'rsi': 2, 'mfi': 2,'ema_10': 1, 'ema_20': 2,'ema_30': 2, 'ema_50': 1,'ema_100': 1, 'sma_10': 1,'sma_20': 2,'sma_30': 2,'sma_50': 1,'sma_100': 1, 'hma_9': 2}
tickers500 = ["AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "ALL", "AMAT", "AMGN", "AMT", "AMZN", "AON", "APA", "APD", "APH", "AVGO", "AZO", "BA", "BAC", "BAX", "BBY", "BDX", "BIIB", "BK", "BKNG", "BMY", "BR", "BSX", "BWA", "BXP", "C", "CAG", "CAH", "CAT", "CB", "CCI", "CCL", "CDNS", "CDW", "CE", "CF", "CFG", "CHD", "CHRW", "CHTR", "CI", "CINF", "CL", "CLX", "CMA", "CMCSA", "CME", "CMG", "CMI", "CMS", "CNC", "COF", "COO", "COP", "COST", "COTY", "CPB", "CPRT", "CRM", "CSCO", "CSX", "CTAS", "CTLT", "CTSH", "CTVA", "CVS", "CVX", "D", "DAL", "DD", "DE", "DFS", "DG", "DGX", "DHI", "DHR", "DIS", "DISH", "DLTR", "DLR", "DOV", "DOW", "DRI", "DTE", "DUK", "DVA", "DVN", "DXCM", "EA", "EBAY", "ECL", "ED", "EMN", "EMR", "ENPH", "EOG", "EQIX", "EQR", "ESS", "ETN", "ETR", "ETSY", "EVRG", "EW", "EXC", "EXPD", "EXPE", "EXR", "F", "FANG", "FAST", "FCX", "FDX", "FE", "FFIV", "FIS", "FISV", "FITB", "FLS", "FLT", "FMC", "FOX", "FOXA", "FRC", "FRT", "FTI", "FTNT", "FTV", "GD", "GE", "GILD", "GIS", "GL", "GLW", "GM", "GNRC", "GOLD", "GOOG", "GOOGL", "GPC", "GPN", "GPS", "GRMN", "GS", "GWW", "HAL", "HAS", "HBAN", "HBI", "HCA", "HD", "HES", "HIG", "HII", "HLT", "HOLX", "HON", "HPE", "HPQ", "HRL", "HSIC", "HST", "HSY", "HUM", "HWM", "IBM", "ICE", "IDXX", "IEX", "IFF", "IGM", "IHS", "ILMN", "INCY", "INTC", "INTU", "IP", "IPG", "IPGP", "IQV", "IR", "IRM", "ISRG", "IT", "ITW", "IVZ", "J", "JBHT", "JCI", "JKHY", "JNJ", "JNPR", "JPM", "K", "KEY", "KEYS", "KHC", "KIM", "KLAC", "KMB", "KMI", "KMX", "KO", "KR", "L", "LDOS", "LEG", "LEN", "LH", "LHX", "LIN", "LKQ", "LLY", "LMT", "LNC", "LNT", "LOW", "LRCX", "LUMN", "LUV", "LVS", "LW", "LYB", "LYV", "MA", "MAA", "MAR", "MAS", "MCD", "MCHP", "MCK", "MCO", "MDLZ", "MDT", "MET", "MGM", "MHK", "MKC", "MKTX", "MLM", "MMC", "MMM", "MNST", "MO", "MOS", "MPC", "MPWR", "MRK", "MRO", "MS", "MSCI", "MSFT", "MTB", "MTD", "MU", "NCLH", "NDAQ", "NEE", "NEM", "NFLX", "NI", "NKE", "NKTR", "NOC", "NOV", "NOW", "NRG", "NSC", "NTAP", "NTRS", "NUE", "NVDA", "NVR", "NWL", "NWS", "NWSA", "O", "ODFL", "OKE", "OMC", "ON", "ORCL", "ORLY", "OXY", "PAA", "PACW", "PAG", "PANW", "PAYC", "PAYX", "PCAR", "PCG", "PEAK", "PEG", "PEP", "PFE", "PFG", "PG", "PGR", "PH", "PHM", "PKG", "PKI", "PLD", "PM", "PNC", "PNR", "PNW", "POOL", "PPG", "PPL", "PRGO", "PRU", "PSA", "PSX", "PTC", "PVH", "PWR", "PXD", "PYPL", "QCOM", "QRVO", "RCL", "RE", "REG", "REGN", "RF", "RHI", "RJF", "RL", "RMD", "ROK", "ROL", "ROP", "ROST", "RSG", "RTX", "SBAC", "SBUX", "SCHW", "SEE", "SHW", "SJM", "SLB", "SLG", "SNA", "SNPS", "SO", "SPG", "SPGI", "SRE", "STE", "STT", "STX", "STZ", "SWK", "SWKS", "SYF", "SYK", "SYY", "T", "TAP", "TDG", "TDY", "TEL", "TER", "TFC", "TFX", "TGT", "TJX", "TMO", "TMUS", "TPR", "TRMB", "TROW", "TRV", "TSCO", "TSN", "TT", "TTWO", "TXN", "TXT", "TYL", "UA", "UAA", "UAL", "UDR", "UHS", "ULTA", "UNH", "UNM", "UNP", "UPS", "URI", "USB", "V", "VFC", "VLO", "VMC", "VNO", "VRSK", "VRSN", "VRTX", "VTR", "VTRS", "VZ", "WAB", "WAT", "WBA", "WDC", "WEC", "WELL", "WFC", "WHR", "WM", "WMB", "WMT", "WRB", "WRK", "WST", "WU", "WY", "WYNN", "XEL", "XOM", "XRAY", "XYL", "YUM", "ZBH", "ZBRA", "ZION", "ZTS" ]
window_sizes_knn = [10, 20, 30, 50, 100]

In [4]:
# Memoria
status = pd.DataFrame({'Stock': tickers500, 'Previous_Status':[0] *len(tickers500),'Status':[0] *len(tickers500), 'Cycle':[0] *len(tickers500), 'Price':[0] *len(tickers500), 'Quantity_Money':[0] *len(tickers500), 'Quantity_Stocks':[0] *len(tickers500), 'Profit':[0] *len(tickers500)}) 
#status.to_csv("/content/drive/MyDrive/status.csv")
movement = pd.DataFrame({'Date': [], 'Buy': [], 'Sell': [], 'Cumulative Profit': [] })
#movement.to_csv("/content/drive/MyDrive/movement.csv")

In [5]:
# Memoria Recurse
movement = pd.read_csv("/content/drive/MyDrive/movement.csv")
status = pd.read_csv("/content/drive/MyDrive/status.csv")

In [6]:
# Defined Functions
def download(ticker):
    start_date = datetime.now() - timedelta(days=250)
    ticker_data = yf.download(ticker, start=start_date, interval="1D") 
    ticker_data = pd.DataFrame(ticker_data)

    return ticker_data

def calculate_TA(ticker_data_1):
              
    ticker_data_2 = ticker_data_1.copy()

    ticker_data_2['rsi'] = rsi.relative_strength_index(ticker_data_2['Close'], 14)
    ticker_data_2['mfi'] = mfi.money_flow_index(ticker_data_2['High'], ticker_data_1['Low'], ticker_data_2['Close'], ticker_data_2['Volume'], 14)
    ticker_data_2['upper_band'], ticker_data_2['middle_band'], ticker_data_2['lower_band'] = bb.upper_bollinger_band(ticker_data_2['Close'], 20), bb.middle_bollinger_band(ticker_data_2['Close'], 20), bb.lower_bollinger_band(ticker_data_2['Close'], 20)
    ticker_data_2['sma_10'] = sma.simple_moving_average(ticker_data_2['Close'], 10)
    ticker_data_2['sma_20'] = sma.simple_moving_average(ticker_data_2['Close'], 20)
    ticker_data_2['sma_30'] = sma.simple_moving_average(ticker_data_2['Close'], 30)
    ticker_data_2['sma_50'] = sma.simple_moving_average(ticker_data_2['Close'], 50)
    ticker_data_2['sma_100'] = sma.simple_moving_average(ticker_data_2['Close'], 100)
    ticker_data_2['ema_10'] = ticker_data_2['Close'].ewm(span=10, adjust=True).mean()
    ticker_data_2['ema_20'] = ticker_data_2['Close'].ewm(span=20, adjust=True).mean()
    ticker_data_2['ema_30'] = ticker_data_2['Close'].ewm(span=30, adjust=True).mean()
    ticker_data_2['ema_50'] = ticker_data_2['Close'].ewm(span=50, adjust=True).mean()
    ticker_data_2['ema_100'] = ticker_data_2['Close'].ewm(span=100, adjust=True).mean()
    ticker_data_2['hma_9'] = hma.hull_moving_average(ticker_data_2['Close'], 9)

    ticker_data_2['ema_10'].iloc[:10] = np.nan
    ticker_data_2['ema_20'].iloc[:20] = np.nan
    ticker_data_2['ema_30'].iloc[:30] = np.nan
    ticker_data_2['ema_50'].iloc[:50] = np.nan
    ticker_data_2['ema_100'].iloc[:100] = np.nan
    
    ticker_data_3 = ticker_data_2.dropna()

    ticker_data_3['TA_Buy'] = ticker_data_3.apply(lambda row: sum([
        weights['rsi'] * (row['rsi'] < 30),
        weights['mfi'] * (row['mfi'] < 20),
        weights['upper_band'] * (row['Close'] < row['upper_band']),
        weights['middle_band'] * (row['Close'] < row['middle_band']),
        weights['lower_band'] * (row['Close'] <= row['lower_band']),
        weights['sma_10'] * (row['Close'] > row['sma_10']),
        weights['sma_20'] * (row['Close'] > row['sma_20']),
        weights['sma_30'] * (row['Close'] > row['sma_30']),
        weights['sma_50'] * (row['Close'] > row['sma_50']),
        weights['sma_100'] * (row['Close'] > row['sma_100']),
        weights['ema_10'] * (row['Close'] > row['ema_10']),
        weights['ema_20'] * (row['Close'] > row['ema_20']),
        weights['ema_30'] * (row['Close'] > row['ema_30']),
        weights['ema_50'] * (row['Close'] > row['ema_50']),
        weights['ema_100'] * (row['Close'] > row['ema_100']),
        weights['hma_9'] * (row['Close'] > row['hma_9'])]), axis=1)
    
    ticker_data_3['TA_Sell'] = ticker_data_3.apply(lambda row: sum([
        weights['rsi'] * (row['rsi'] > 70),
        weights['mfi'] * (row['mfi'] > 80),
        weights['upper_band'] * (row['Close'] >= row['upper_band']),
        weights['middle_band'] * (row['Close'] > row['middle_band']),
        weights['lower_band'] * (row['Close'] > row['lower_band']),
        weights['sma_10'] * (row['Close'] < row['sma_10']),
        weights['sma_20'] * (row['Close'] < row['sma_20']),
        weights['sma_30'] * (row['Close'] < row['sma_30']),
        weights['sma_50'] * (row['Close'] < row['sma_50']),
        weights['sma_100'] * (row['Close'] < row['sma_100']),
        weights['ema_10'] * (row['Close'] < row['ema_10']),
        weights['ema_20'] * (row['Close'] < row['ema_20']),
        weights['ema_30'] * (row['Close'] < row['ema_30']),
        weights['ema_50'] * (row['Close'] < row['ema_50']),
        weights['ema_100'] * (row['Close'] < row['ema_100']),
        weights['hma_9'] * (row['Close'] < row['hma_9'])]), axis=1)                                                    

    return ticker_data_3

def calculate_KNN(ticker_data_3):
    
    ticker_data_4 = ticker_data_3.copy()
    count_sell = 0
    count_buy = 0
    
    for window_size in window_sizes_knn:
        X = ticker_data_4['Close'].values
        k = 3
        knn = KNeighborsRegressor(n_neighbors=k)
        knn.fit(X.reshape(-1, 1), X)
        X_pred = X[-window_size:].copy()
        for i in range(5):
            y_pred = knn.predict(X_pred[-window_size:].reshape(-1, 1))
            X_pred = np.append(X_pred, y_pred)
        if max(X_pred[window_size:window_size+5]) < 1.15*X[-1]:
            count_buy += 1
        if min(X_pred[window_size:window_size+5]) < 0.85*X[-1]:
            count_sell += 1
    
    ticker_data_4['KNN_Buy'] = count_buy
    ticker_data_4['KNN_Sell'] = count_sell
    ticker_data_4['Total_Buy'] = count_buy + ticker_data_4['TA_Buy']
    ticker_data_4['Total_Sell'] = count_sell + ticker_data_4['TA_Sell']
    
    ticker_data_4['Total'] = (ticker_data_4['TA_Buy'] + ticker_data_4['KNN_Buy']) - (ticker_data_4['TA_Sell'] + ticker_data_4['KNN_Sell'])
    ticker_data_4 = ticker_data_4[["Close", "TA_Buy", "KNN_Buy", "Total_Buy", "TA_Sell", "KNN_Sell", "Total_Sell", "Total"]]
    
    return(ticker_data_4)


def macd_signal(ticker_data_4):
    
    fastLength = 12
    slowLength = 26
    signalLength = 9
    ema_fast = ticker_data_4['Close'].ewm(span=fastLength, adjust=False).mean()
    ema_slow = ticker_data_4['Close'].ewm(span=slowLength, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signalLength, adjust=False).mean()
    hist = macd - signal

    # Find the last signal
    last_signal = None
    if macd.iloc[-1] > signal.iloc[-1]:
        last_signal = 'Buy'
    elif macd.iloc[-1] < signal.iloc[-1]:
        last_signal = 'Sell'
    else:
        last_signal = None

    return last_signal

def email(indications):
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_username = "your_email@gmail.com"
    smtp_password = "password"
    smtp_connection = smtplib.SMTP(smtp_server, smtp_port)
    smtp_connection.starttls()
    smtp_connection.login(smtp_username, smtp_password)

    from_address = "your_email@gmail.com"
    to_address = "your_email@gmail.com"
    subject = "Stock Indications " + str(datetime.now())
    body = indications
    message = MIMEMultipart()
    message["From"] = from_address
    message["To"] = to_address
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    # Send the email and close
    smtp_connection.sendmail(from_address, to_address, message.as_string())
    smtp_connection.quit()

In [7]:
# Run the main code every minute
text = "The indications are: "
buy_text = "Buy: "
sell_text = "Sell: "
b_text = " "
s_text = " "
min_profit = 0
inicial_inv = 100
movement = pd.read_csv("/content/drive/MyDrive/movement.csv")
movement = movement[['Date', 'Buy', 'Sell', 'Cumulative Profit']]
        
for i, ticker in enumerate(tickers500):
    status = pd.read_csv("/content/drive/MyDrive/status.csv")
    status = status[["Stock", "Cycle","Profit", "Price","Status", "Quantity_Money","Quantity_Stocks"]]
    
    ticker_data_1 = ""
    ticker_data_2 = ""
    ticker_data_3 = ""
    ticker_data_4 = ""
    ticker_data_5 = ""
    
    ticker_data_1 = download(ticker)
    
    if not ticker_data_1.empty and len(ticker_data_1) >= 45:
          Price = ticker_data_1['Open'].iloc[-1]
          action = macd_signal(ticker_data_1)
          ticker_data_2 = calculate_TA(ticker_data_1)
          ticker_data_3 = calculate_KNN(ticker_data_2)
          ticker_data_4 = ticker_data_3.tail(1) 

          ticker_data_5 = ticker_data_3.drop(ticker_data_3.index[-1])
          previous_value = macd_signal(ticker_data_5)
                                      
          cycle = status.loc[status["Stock"] == ticker, "Cycle"].iloc[0]   
          ind_profit = status.loc[status["Stock"] == ticker, "Profit"].iloc[0]
          status_price = status.loc[status["Stock"] == ticker, "Price"].iloc[0]
          status_value = status.loc[status["Stock"] == ticker, "Status"].iloc[0]
          quantity = status.loc[status["Stock"] == ticker, "Quantity_Money"].iloc[0]   
          quantity_stocks = status.loc[status["Stock"] == ticker, "Quantity_Stocks"].iloc[0]

          if  ticker_data_4["Total_Buy"].iloc[0] >= 15 and status_value == 0 and action == "Buy" and previous_value == "Sell":
              status.loc[status["Stock"] == ticker, "Status"] = 1
              status.loc[status["Stock"] == ticker, "Price"] = Price
              status.loc[status["Stock"] == ticker, "Cycle"] = cycle + 1
              status.loc[status["Stock"] == ticker, "Quantity_Money"] = inicial_inv + ind_profit
              status.loc[status["Stock"] == ticker, "Quantity_Stocks"] = (inicial_inv + ind_profit) / Price
              b_text = b_text + " S: " + ticker + " P: " + str(ticker_data_4["Total_Buy"].iloc[0]) + " $: " + str(inicial_inv + ind_profit)
              
          elif ticker_data_4["Total_Sell"].iloc[0] <= -15 and status_value == 1 and action == "Sell" and previous_value == "Buy":
              status.loc[status["Stock"] == ticker, "Price"] = 0
              status.loc[status["Stock"] == ticker, "Status"] = 0
              status.loc[status["Stock"] == ticker, "Quantity_Money"] = 0
              status.loc[status["Stock"] == ticker, "Quantity_Stocks"] = 0

              ind_profit = ind_profit + (Price - status_price) * 0.9995 * quantity_stocks
              status.loc[status["Stock"] == ticker, "Profit"] = ind_profit
              s_text = s_text +" S:" + ticker + " P: " + str(ticker_data_4["Total_Sell"].iloc[0])

          status.to_csv("/content/drive/MyDrive/status.csv")
          clear_output()

# Movement Csv
Profit = status['Profit'].sum()
nyse_time_zone = pytz.timezone('America/New_York')
current_time = datetime.now(nyse_time_zone)
movement_row = {'Date':current_time, 'Buy': b_text, 'Sell': s_text, 'Profit': min_profit, 'Cumulative Profit': Profit}
movement.loc[len(movement)] = movement_row
movement.to_csv("/content/drive/MyDrive/movement.csv")

# Email
Profit = status['Profit'].sum()
buy_text = buy_text + b_text 
sell_text = sell_text + s_text
text = text + buy_text + " " + sell_text + " Profit: " + str(Profit) 
email(text) 

In [8]:
movement

,Date,Buy,Sell,Cumulative Profit
0,2023-04-14 01:08:57.865677-04:00,S: CDNS P: 19 $: 100 S: CHRW P: 19 $: 100 S:...,,0
1,2023-04-14 17:14:13.123283-04:00,S: DXCM P: 21 $: 100 S: LEN P: 21 $: 100 S: ...,,0
2,2023-04-17 17:41:04.811485-04:00,S: AMZN P: 23 $: 100 S: FAST P: 23 $: 100 S:...,,0


In [9]:
text

'The indications are: Buy:   S: AMZN P: 23 $: 100 S: FAST P: 23 $: 100 S: PH P: 17 $: 100 S: TEL P: 22 $: 100 S: WYNN P: 23 $: 100 Sell:   Profit: 0'